In [55]:
from dotenv import load_dotenv

load_dotenv(
    "/home/rajeshgupta/Git_Clones/esrs_gap_analysis/.env"
)

True

In [56]:
import pandas as pd

# Load the Excel file
df = pd.read_excel(
    "./esrs_extraction_results_all_files_sheets_hyperlink_top1.xlsx",
    sheet_name=None,
)

print(f"Loaded Excel file with sheets: {list(df.keys())}")

Loaded Excel file with sheets: ['ESRS 2']


In [57]:
df_esrs = df["ESRS 2"]
df_esrs.head()

,question,requirements,Public Document Content,document_score,ESRS Document Content,requirements_score,Original Hyperlink
0,Has the sustainability statement been prepared...,(BP-1_01) Basis for preparation of sustainabil...,There is an increasing fofocus frfrom regulato...,0.514692,5. The undertaking shall disclose the followin...,0.602468,https://xbrl.efrag.org/e-esrs/esrs-set1-2023.h...
1,Does the undertaking prepare the sustainabilit...,(BP-1_02) Scope of consolidation of consolidat...,There is an increasing fofocus frfrom regulato...,0.513751,5. The undertaking shall disclose the followin...,0.608084,https://xbrl.efrag.org/e-esrs/esrs-set1-2023.h...
2,Does the undertaking describe management and t...,(GOV-1_10) Description of management's role in...,We are committed to sound governance and overs...,0.674317,22. The undertaking shall disclose the followi...,0.588440,https://xbrl.efrag.org/e-esrs/esrs-set1-2023.h...
3,Does the undertaking disclose a description of...,(SBM-1_05) Description of products and service...,"Similarly, y, w ith our registration as an int...",0.744239,consumers and/ or end-users of services that p...,0.518750,https://xbrl.efrag.org/e-esrs/esrs-set1-2023.h...
4,Does the sustainability statement cover the un...,(BP-1_04) Disclosure of extent to which sustai...,There is an increasing fofocus frfrom regulato...,0.508206,5. The undertaking shall disclose the followin...,0.634571,https://xbrl.efrag.org/e-esrs/esrs-set1-2023.h...


In [24]:
# from openai import OpenAI
# import json

# client = OpenAI()


# def get_assessment_for_row(
#     question, requirement, retrieved_content_question, retrieved_content_requirements
# ):

#     n1 = "\n\n"
#     prompt = f"""
#     You are a sustainability reporting expert analyzing the company's ESG disclosures based on the European Sustainability Reporting Standards (ESRS).

#     1. **Question**: {question}
#     2. **ESRS Requirement**: {requirement + n1 + retrieved_content_requirements}
#     3. **Retrieved Content from publicly available company disclosures**: {retrieved_content_question}
#     Your task is to assess the company's readiness to meet the ESRS requirements based on the provided content from the company's disclosures. You need to evaluate whether the company has fully addressed the requirement, partially addressed it, or not addressed it at all.
#     Please provide the following information in the following JSON format:

#     ```json
#     {{
#         "readiness": "Yes/No/Partially",
#         "gap_overview": "<brief description of missing information or gaps>",
#         "recommendations": "<clear actionable recommendations>"
#     }}
#     ```

#     Where:
#     - **readiness**: Return "Yes" if the company has fully addressed the requirement, "No" if the requirement is not addressed at all or is not present in the retrieved disclosed content, and "Partially" if it is partially addressed or the disclosure is incomplete but provides some information.
#     - **gap_overview**: Provide a brief description of any gaps or missing information in the company's disclosure, comparing it to the expected requirements.
#     - **recommendations**: Offer clear and actionable recommendations for the company to improve its sustainability disclosure, based on the gaps identified.

#     Make sure the answers are clear, precise, and formatted in JSON. Ensure that each key in the JSON response is correctly populated.
#     """

#     # Make an OpenAI API call with the prompt
#     response = client.responses.create(
#         model="gpt-4.1-nano",  # You can use the model of your choice
#         input=prompt,
#     )

#     # Get the output from the model
#     output = (
#         response.output[0]
#         .content[0]
#         .text.strip()
#         .replace("```json", "")
#         .replace("```", "")
#     )

#     # Parse the JSON output
#     try:
#         result = json.loads(output)
#     except json.JSONDecodeError:
#         print(f"Error decoding JSON: {output}")
#         result = {
#             "readiness": "No",
#             "gap_overview": "Invalid JSON response",
#             "recommendations": "No recommendations provided",
#         }

#     return result

In [71]:
from openai import OpenAI
import json


client = OpenAI()

def get_assessment_for_row(
    question, requirement, retrieved_content_question, retrieved_content_requirements
):

    n1 = "\n\n"
    prompt = f"""
    **Objective:** Perform a GAP analysis for a specific sustainability disclosure requirement based on provided information. Compare the company's current public disclosures against the official requirement details and generate an assessment of readiness, identify the gaps, and provide actionable recommendations.
    **Context:**
    You are acting as an expert sustainability reporting analyst. Your task is to evaluate a company's compliance with a specific disclosure requirement, often derived from frameworks like the European Sustainability Reporting Standards (ESRS). A GAP analysis involves comparing the 'current state' (what the company discloses) with the 'desired state' (what the standard requires).
    You will receive the following pieces of information for each analysis point:

    1.  `Question`: A specific question addressing a particular aspect of the company's sustainability reporting.
        * Example: "Does the undertaking describe management and the AMS bodies' role in assessing and managing impacts, risks, and opportunities?"
    2.  `Disclosure Requirement`: The reference code or identifier for the specific requirement in the sustainability reporting standard (e.g., ESRS GOV-1_10).
        * Example: "(GOV-1_10) Description of management's role in governance processes"
    3.  `Retrieved Disclosure Requirement Content`: The detailed text from the official standard or regulation explaining *exactly* what needs to be disclosed for this requirement. This defines the 'desired state'.
        * Example: "The ESRS requires companies to provide specific details about how management’s role in ESG governance is structured and the processes used to manage sustainability-related impacts, risks, and opportunities."
    4.  `Retrieved Content from publicly available company disclosures`: The actual text or summary of information found in the company's reports (sustainability reports, annual reports, website, etc.) that relates to the `Question`. This represents the 'current state'.
        * Example: "The company provides general descriptions of how management assesses risks but does not mention sustainability-related risks explicitly. The Nominating and Corporate Governance Committee Charter mentions oversight of ESG programs."


    **Task:**
    Based *only* on the provided input data (`Retrieved Disclosure Requirement Content` and `Retrieved Content from publicly available company disclosures`), perform a GAP analysis by determining the following three components:
    1.  **Readiness:** Assess the company's preparedness to meet the specific disclosure requirement based *solely* on the retrieved content.
        * **"Yes"**: The `Retrieved Content from publicly available company disclosures` fully addresses all aspects mentioned in the `Retrieved Disclosure Requirement Content` and directly answers the `Question`.
        * **"No"**: The `Retrieved Content from publicly available company disclosures` is missing entirely, is irrelevant to the `Question`/`Requirement Content`, or clearly fails to address the core aspects of the requirement. This includes cases where the disclosure doesn't exist yet (e.g., "Sustainability statement does not yet exist").
        * **"Partially"**: The `Retrieved Content from publicly available company disclosures` addresses some aspects of the `Retrieved Disclosure Requirement Content` but is incomplete, lacks necessary detail, is ambiguous, or fails to cover all key elements required.

    2.  **Overview of GAP:** Provide a brief, specific description of the discrepancies or alignment between the `Retrieved Content from publicly available company disclosures` and the `Retrieved Disclosure Requirement Content`.
        * If Readiness is "Yes", briefly state that the requirement appears met based on the provided text.
        * If Readiness is "No" or "Partially", clearly state *what* is missing, incomplete, unclear, or non-compliant in the company's content when compared to the requirement's details. Avoid generic statements; pinpoint the specific information deficit.
        * Example (for "No"): "The company content describes general risk management but lacks specific details on how *sustainability* impacts, risks, and opportunities are assessed by management, as required."
        * Example (for "Partially"): "While oversight by a committee is mentioned, the specific *processes* used by management for assessment and management, as required by the standard, are not detailed."
        * Example (for "Yes"): "The company's disclosure describes the management's role and oversight committee, aligning with the requirement's core elements based on the provided text."

    3.  **Recommendations:** Provide clear, concise, and actionable recommendations for the company to bridge the identified gaps and achieve full compliance with the disclosure requirement.
        * Recommendations should directly address the issues identified in the `Overview of GAP`.
        * Focus on *what* the company needs to *do* or *disclose*.
        * If Readiness is "Yes", the recommendation can be brief, suggesting maintaining the current level of detail or considering enhancements for clarity.
        * Example (for "No"/"Partially"): "Enhance the disclosure to explicitly detail the processes management uses to assess and manage sustainability-related impacts, risks, and opportunities. Specify how this integrates with overall governance and risk management frameworks. Consider outlining the frequency and nature of management reviews related to ESG."
        * Example (for "Yes"): "Maintain the current level of detail. Consider adding a diagram illustrating the governance structure for ESG oversight to further enhance clarity."


    **Output Format:**
    Please provide the output in the following JSON format:

    ```json
    {{
        "readiness": "Yes/No/Partially", 
        "gap_overview": "<brief description of missing information or gaps, or confirmation of alignment>",
        "recommendations": "<clear actionable recommendations to bridge the gap, or suggestion to maintain/enhance if gap is minimal/none>"
    }}
    ```
    BEGIN ANALYSIS:
    Input Data:
    1. **Question**: {question}
    2. Disclosure Requirement: {requirement}
    3. **Retrieved Disclosure Requirement Content**: {retrieved_content_requirements}
    4. **Retrieved Content from publicly available company disclosures**: {retrieved_content_question}
    END ANALYSIS
    (Now, generate the JSON output based on the analysis of the provided inputs)
    """

    # Make an OpenAI API call with the prompt
    response = client.responses.create(
        model="gpt-4.1-nano",  # You can use the model of your choice
        input=prompt,
    )

    # Get the output from the model
    output = (
        response.output[0]
        .content[0]
        .text.strip()
        .replace("```json", "")
        .replace("```", "")
    )

    # Parse the JSON output
    try:
        result = json.loads(output)
    except json.JSONDecodeError:
        print(f"Error decoding JSON: {output}")
        result = {
            "readiness": "No",
            "gap_overview": "Invalid JSON response",
            "recommendations": "No recommendations provided",
        }

    return result

In [72]:
def apply_assessment(df):
    readiness_list = []
    gap_overview_list = []
    recommendations_list = []

    for idx, row in df.iterrows():
        result = get_assessment_for_row(
            row["question"],
            row["requirements"],
            row["Public Document Content"],
            row["ESRS Document Content"],
        )
        readiness_list.append(result.get("readiness", "No"))
        gap_overview_list.append(result.get("gap_overview", ""))
        recommendations_list.append(result.get("recommendations", ""))

    df["Readiness"] = readiness_list
    df["Overview of GAP"] = gap_overview_list
    df["Recommendations"] = recommendations_list

    return df

In [30]:
updated_dfs = {}
for sheet_name, sheet_df in df.items():
    print(f"Processing sheet: {sheet_name}")
    updated_dfs[sheet_name] = apply_assessment(sheet_df)

# Save all processed sheets into one Excel file
output_path = "./esrs_extraction_results_all_sheets_assessment_hyperlink.xlsx"
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    for sheet_name, sheet_df in updated_dfs.items():
        sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Saved processed file to {output_path}")

Processing sheet: ESRS 2


KeyError: 'Pubic Document Content'

In [73]:

df_esrs = apply_assessment(df_esrs)


Error decoding JSON: {
    "readiness": "No",
    "gap_overview": "The company's disclosure primarily discusses ESG risks related to regulation, reputation, and operational risks but does not explicitly address whether the company has used or is considering the use of an exemption or option to omit specific information related to intellectual property, know-how, or innovation as outlined in the requirement. It lacks specific mention of how such options or exemptions are applied or documented, which is essential for full compliance.",
    "recommendations": "The company should explicitly disclose whether it has utilized any options or exemptions to omit specific information about intellectual property, know-how, or innovation, including details on the scope and reasons for such decisions. It should also document its process for applying these options in accordance with ESRS guidelines, ensuring transparency and completeness in its disclosures."
Error decoding JSON: {
  "readiness": "No"

In [74]:
df_esrs.to_excel(
    "./esrs_extraction_results_esrs_docling_assessment_hyperlink_nano_model_top1_new.xlsx",
    index=False,
)

In [ ]:

import openpyxl

def extract_hyperlinks(file_path, sheet_name, column_letter):
    workbook = openpyxl.load_workbook(file_path, data_only=True)
    sheet = workbook[sheet_name]

    hyperlinks = []

    for row in sheet.iter_rows(min_row=2):  # Skip header
        cell = row[openpyxl.utils.column_index_from_string(column_letter) - 1]
        if cell.hyperlink:
            hyperlinks.append((cell.value, cell.hyperlink.display))

    return hyperlinks


# Example usage
file_path =     "/home/rajeshgupta/Downloads/EFRAG IG 3 List of ESRS Data Points (1).xlsx"

column_letter = "F"  # Adjust this to your hyperlink column

results = extract_hyperlinks(file_path, sheet_name, column_letter)

for text, link in results:
    print(f"Text: {text}\nLink: {link}\n")




Text: Basis for preparation of sustainability statement
Link: https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4108

Text: Scope of consolidation of consolidated sustainability statement is same as for financial statements
Link: https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4112

Text: Indication of subsidiary undertakings included in consolidation that are exempted from individual or consolidated sustainability reporting 
Link: https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4114

Text: Disclosure of extent to which sustainability statement covers upstream and downstream value chain 
Link: https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4116

Text: Option to omit specific piece of information corresponding to intellectual property, know-how or results of innovation has been used
Link: https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4118

Text: Option allowed by Member State to omit disclosure of impending developments or matters in course of negotiation has been u

In [ ]:
import openpyxl
import re

def extract_id_from_text(text):
    """Extract content inside parentheses like (GOV-1_10) from text"""
    if not text:
        return None
    match = re.search(r'\((.*?)\)', text)
    if not match:
        return None

    content = match.group(1)
    return content.split('.')[-1] 

def get_hyperlink_for_id(id_value, esrs_sheet):
    """Find hyperlink for given ID in ESRS sheet"""
    # Assuming:
    # - ID is in column A (index 0)
    # - Hyperlink is in column F (index 5)
    # - Headers are in row 2, data starts from row 3
    for row in esrs_sheet.iter_rows(min_row=3, values_only=False):
        esrs_id = row[0].value  # Column A (ID)
        name_cell = row[5]      # Column F (Name with hyperlink)
        
        if esrs_id == id_value:
            if name_cell.hyperlink:
                return name_cell.hyperlink.display  # Use .target for the actual URL
    return None

def add_hyperlinks_to_disclosure_requirements(file1_path, file2_path):
    # Load the first file (contains the disclosure requirements)
    wb1 = openpyxl.load_workbook(file1_path)
    
    # Load the second file (contains the ESRS 2 sheet)
    wb2 = openpyxl.load_workbook(file2_path, keep_vba=True)
    esrs_sheet = wb2['ESRS 2']  # Assuming the sheet name is 'ESRS 2'

    for sheet_name in wb1.sheetnames:
        sheet1 = wb1[sheet_name]
        
        # Find or create Hyperlink column
        headers = [cell.value for cell in sheet1[1]]
        if "Hyperlink" not in headers:
            hyperlink_col_idx = len(headers) + 1
            sheet1.cell(row=1, column=hyperlink_col_idx, value="Hyperlink")
        else:
            hyperlink_col_idx = headers.index("Hyperlink") + 1

        # Process each row (starting from row 2)
        for row_idx in range(2, sheet1.max_row + 1):
            disclosure_text = sheet1.cell(row=row_idx, column=3).value  # Assuming column B
            
            if disclosure_text:
                extracted_id = extract_id_from_text(disclosure_text)
                if extracted_id:
                    # First try ESRS 2 sheet
                    esrs_sheet = wb2.get('ESRS 2', None)
                    hyperlink = None
                    
                    if esrs_sheet:
                        hyperlink = get_hyperlink_for_id(extracted_id, esrs_sheet)
                    
                    # If not found in ESRS 2, search all other sheets
                    if not hyperlink:
                        for sheet in wb2.sheetnames:
                            if sheet != 'ESRS 2':  # Skip ESRS 2 as we already checked it
                                current_sheet = wb2[sheet]
                                hyperlink = get_hyperlink_for_id(extracted_id, current_sheet)
                                if hyperlink:
                                    break
                    sheet1.cell(row=row_idx, column=hyperlink_col_idx, value=hyperlink)
    
    # Save the modified first file
    wb1.save(file1_path)

# Example usage
file1_path = "/home/rajeshgupta/Git_Clones/esrs_gap_analysis/Download_ESRS GAP_Twilio (1).xlsx"
file2_path = "/home/rajeshgupta/Downloads/EFRAG IG 3 List of ESRS Data Points (1).xlsx"

add_hyperlinks_to_disclosure_requirements(file1_path, file2_path)

https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4108
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4112
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4201
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4273
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4116
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4118
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4299
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4120
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4267
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4269
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4390
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4396
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4382
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4188
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4197
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4199
https://xbrl.efrag.org/e-esrs/esrs-set1-2023.html - 4209
https://xbrl.efrag.org/e-esrs/e

In [9]:
import pprint

pprint.pprint(df_esrs["ESRS Document Content"].iloc[3])

('Disclosure Requirement related to ESRS 2 SBM-3 – Material impacts, risks and '
 'opportunities and their interaction with strategy and business model\n'
 '\n'
 'AR 5. Impacts on consumers and/or end-users can originate in the '
 'undertaking’s business model or strategy in a number of different ways. For '
 'example, impacts may relate to the undertaking’s value proposition (for '
 'example, providing online platforms with potential for online and offline '
 'harm), its value chain (for example, speed in developing products or '
 'services, or delivering projects, with risks to health and safety), or its '
 'cost structure and the revenue model (for example, sales-maximising '
 'incentives that put consumers at risk).\n'
 '\n'
 'Disclosure Requirement related to ESRS 2 SBM-3 – Material impacts, risks and '
 'opportunities and their interaction with strategy and business model\n'
 '\n'
 'AR 5. Impacts on consumers and/or end-users can originate in the '
 'undertaking’s business model 

In [9]:
pprint.pprint(df_esrs["ESRS Document Content"].iloc[0])

('1. Basis for preparation\n'
 '\n'
 'Disclosure Requirement BP-1 – General basis for preparation of the '
 'sustainability statement\n'
 '\n'
 '3. The undertaking shall disclose the general basis for preparation of its '
 'sustainability statement.\n'
 '\n'
 '4. The objective of this Disclosure Requirement is to provide an '
 'understanding of how the undertaking prepares its sustainability statement , '
 'including the scope of consolidation, the upstream and downstream value '
 'chain information and, where relevant, whether the undertaking has used any '
 'of the options for omitting information referred to in points d) and e) in '
 'the following paragraph.\n'
 '\n'
 '1. Basis for preparation\n'
 '\n'
 'Disclosure Requirement BP-1 – General basis for preparation of the '
 'sustainability statement\n'
 '\n'
 '3. The undertaking shall disclose the general basis for preparation of its '
 'sustainability statement.\n'
 '\n'
 '4. The objective of this Disclosure Requirement is to provi

In [10]:
pprint.pprint(df_esrs["ESRS Document Content"].iloc[5])

('105. The undertaking is not required to disclose classified information or '
 'sensitive information , even if such information is considered material.\n'
 '\n'
 '106. When disclosing information about its strategy, plans and actions , '
 'where a specific piece of information corresponding to intellectual '
 'property, know-how or the results of innovation is relevant to meet the '
 'objective of a Disclosure Requirement, the undertaking may nevertheless omit '
 'that specific piece of information if it: (a) is secret in the sense that it '
 'is not, as a body or in the precise configuration and assembly of its '
 'components, generally known among or readily accessible to persons within '
 'the circles that normally deal with the kind of information in question; (b) '
 'has commercial value because it is secret; and (c) has been subject to '
 'reasonable steps by the undertaking to keep it secret.\n'
 '\n'
 '105. The undertaking is not required to disclose classified information or

In [1]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader(
    "/home/saitejatallam/Agents/Incomecalculation/infrrd-agents-ml/esrs_content_unstructured.txt"
)
documents = loader.load()

In [8]:
import pprint

pprint.pprint(documents[0].page_content)

NameError: name 'documents' is not defined